<a href="https://colab.research.google.com/github/ericbzhou/genai_creativity/blob/main/DINO_V2_embedding_0602.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Load all images into the data folder
import os

# Set the path to the root directory containing the .png files
root_path = "/content/drive/MyDrive/Partial_da_imgs"

# Define a function to recursively load all .png files in a directory into a list
def load_data_in_directory(dir_path):
    data = []
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        if os.path.isdir(file_path):
            # If the file is a directory, recursively load the files inside it
            data += load_data_in_directory(file_path)
        elif file_name.endswith(".png"):
            # If the file is a .png file, load the data and add it to the list
            try:
                data.append(file_path)
            except:
                print(f"Error loading file: {file_path}")
    return data

# Call the function on the root directory to load all .png files in the directory tree
data = load_data_in_directory(root_path)

In [ ]:
# print the first five image paths
print(data[:5])

['/content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/CE3CA729-47DF-0DA9-3E00-9D9D7F36F8D9.png', '/content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/74424080-8D8F-99D4-7A1D-F20501D0BEB6.png', '/content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/A90364AE-282B-8F17-E469-0B83D5883CE9.png', '/content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/79E9D525-FD54-F11D-2F75-94FCB60D9BBD.png', '/content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/28C33EAD-04D8-B44D-8CB3-AF92A9BFDA4C.png']


In [ ]:
print(len(data))

1362


In [ ]:

# create a soft link to make the default CUDA12 to CUDA10)
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda


In [ ]:
# setting up Cuda on GPU
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.3-450.51.06-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.3-450.51.06-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda

--2023-05-28 21:05:49--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2023-05-28 21:05:49 (5.50 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

--2023-05-28 21:05:50--  https://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.3-450.51.06-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 O

In [ ]:
os.environ['PATH'] += '/usr/local/cuda/bin'
os.environ['LD_LIBRARY_PATH'] += '/usr/local/cuda/lib64'


In [ ]:
!wget https://raw.githubusercontent.com/facebookresearch/dinov2/main/requirements.txt
!pwd
!pip install -r requirements.txt

--2023-05-28 21:06:55--  https://raw.githubusercontent.com/facebookresearch/dinov2/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 213 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     213  --.-KB/s    in 0s      

2023-05-28 21:06:55 (13.3 MB/s) - ‘requirements.txt’ saved [213/213]

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 662.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.3 MB/s eta 0:00:00
    

In [ ]:
! pip install xformers==0.0.18

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# checking the pytorch version is compatible with xformers
import torch
print(torch.__version__)

2.0.0+cu117


In [ ]:
import torchvision.transforms as transforms
from PIL import Image 


# Step 1: Load pre-trained DINO v2 model
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14').cuda()



Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitg14/dinov2_vitg14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitg14_pretrain.pth
100%|██████████| 4.23G/4.23G [02:04<00:00, 36.4MB/s]


In [ ]:
# Step 2: Prepare dataset
# Define the transformation pipeline for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a specific size
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [ ]:
model

In [ ]:
# Step 3: Forward pass through the model
# 'data' is the list of image paths

embeddings = []
image_file_path_folder = []
for i, path in enumerate(data):
    try:
      image = Image.open(path)
      image = transform(image).unsqueeze(0)  # Add batch dimension
    except Exception as e:
      print(f"{i} | Error: {e}, File: {path}")
      continue
    with torch.no_grad():
      # move input data to gpu from cpu 
      cpu_tensor = torch.tensor(image)
      gpu_tensor = cpu_tensor.to('cuda')
      try:
        embedding = model(gpu_tensor)
        print(f"{i} | File: {path} {embedding.shape}")
      except Exception as e:
        print(f"{i} | Error: {e}, File: {path}")
    
    embeddings.append(embedding)
    # extract only the file names of the ith embedding
    image_file_path = os.path.basename(data[i])
    image_file_path_folder.append(image_file_path)

<ipython-input-15-3bcd619e26c8>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cpu_tensor = torch.tensor(image)
/usr/local/lib/python3.10/dist-packages/xformers/ops/unbind.py:46: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage_data_ptr = tensors[0].storage().data_ptr()
/usr/local/lib/python3.10/dist-packages/xformers/ops/unbind.py:48: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() inst

0 | File: /content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/CE3CA729-47DF-0DA9-3E00-9D9D7F36F8D9.png torch.Size([1, 1536])
1 | File: /content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/74424080-8D8F-99D4-7A1D-F20501D0BEB6.png torch.Size([1, 1536])
2 | File: /content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/A90364AE-282B-8F17-E469-0B83D5883CE9.png torch.Size([1, 1536])
3 | File: /content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/79E9D525-FD54-F11D-2F75-94FCB60D9BBD.png torch.Size([1, 1536])
4 | File: /content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/28C33EAD-04D8-B44D-8CB3-AF92A9BFDA4C.png torch.Size([1, 1536])
5 | File: /content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/7C42266F-942A-72BC-9743-E014A7A5510D.png torch.Size([1, 1536])
6 | File: /content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/3C2F70FB-FAA6-8A06-C6F5-231088072BF5.png torch.Size([1, 1536])
7 | File: /content/drive/MyDrive/Partial_da_imgs/s/some-ai-images/A9E7681A-EDFB-F61E-5D95-3497313

In [ ]:
# print the length of the embedding list and the first 6 embeddings
print(len(embeddings))
print(embeddings[:5])

1328
[tensor([[-1.4964, -0.0447,  2.7404,  ...,  0.8794,  1.0066,  1.5334]],
       device='cuda:0'), tensor([[-0.0316,  2.3656, -2.4551,  ...,  2.2011,  0.5091,  0.6325]],
       device='cuda:0'), tensor([[-2.1922, -0.7851,  3.0302,  ...,  0.1090,  0.5440,  0.5211]],
       device='cuda:0'), tensor([[-0.5942,  1.3745,  1.4984,  ..., -1.6838,  1.1860, -1.7795]],
       device='cuda:0'), tensor([[ 0.0776,  0.1157,  0.5414,  ..., -0.0961, -2.6756,  0.1706]],
       device='cuda:0')]


In [ ]:
print(len(image_file_path_folder))
print(image_file_path_folder[:5])

1328
['CE3CA729-47DF-0DA9-3E00-9D9D7F36F8D9.png', '74424080-8D8F-99D4-7A1D-F20501D0BEB6.png', 'A90364AE-282B-8F17-E469-0B83D5883CE9.png', '79E9D525-FD54-F11D-2F75-94FCB60D9BBD.png', '28C33EAD-04D8-B44D-8CB3-AF92A9BFDA4C.png']


In [ ]:
# Convert the list of tensors to a list of NumPy arrays on CPU
import numpy as np
embeddings_np = [emb.cpu().numpy() for emb in embeddings]

print(embeddings_np[:4])


[array([[-1.4964095 , -0.04474308,  2.7404401 , ...,  0.879358  ,
         1.0065595 ,  1.5333941 ]], dtype=float32), array([[-0.03163963,  2.3655646 , -2.4551234 , ...,  2.2011232 ,
         0.50913507,  0.63248694]], dtype=float32), array([[-2.1921668 , -0.78510344,  3.0302246 , ...,  0.10895744,
         0.5440234 ,  0.521077  ]], dtype=float32), array([[-0.594219 ,  1.3744822,  1.4983829, ..., -1.6837863,  1.1859794,
        -1.77951  ]], dtype=float32)]


In [ ]:
# Specify the path and folder name
path = "/content"
folder_name = "DINO_embs"

# Create the full path
full_path = os.path.join(path, folder_name)

# Create the folder
os.makedirs(full_path, exist_ok=True)

In [ ]:
# Save the list of NumPy arrays to .npy files using the specified file path names in the DINO_embs folder
folder_path = "/content/DINO_embs"
for i, emb_np in enumerate(embeddings_np):
    file_path = os.path.join(folder_path, f"{image_file_path_folder[i]}.npy")
    np.save(file_path, emb_np)



In [ ]:
# Create a zip file of the saved embeddings
import shutil
shutil.make_archive("/content/a/embeddings_Partial_da_imgs_V2", 'zip', folder_path)

import shutil

# Specify the path of the zip file
zip_file_path = "/content/a/embeddings_Partial_da_imgs_V2.zip"


# Move the zip file to the content folder
shutil.move(zip_file_path, "/content/a/embeddings_Partial_da_imgs_V2.zip")

'/content/a/embeddings_Partial_da_imgs_V2.zip'

In [ ]:
# download the zip file
#from google.colab import files

#files.download("/content/a/embeddings_Partial_da_imgs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>